# 概要

![jupyter](./思维导图.png)

- 基于上述思维导图，案例分析顺序为‘车金融项目重要指标概览’、‘渠道名单数量分析’、‘渠道名单质量分析’、‘通话时长分析’、‘通话次数分析’、‘风控周期波动性分析’、‘直属门店分析’；
- 此篇记录详尽记录代码细节及分析思路；


# 一、数据加载及清洗

## 数据加载&预览

In [ ]:
import pandas as pd # 数据处理包
import numpy as np # 向量、矩阵支持包
import matplotlib.pyplot as plt # 绘图包

df = pd.read_excel(r'D:\WD53259\数据\2019年7月\借呗京东数据每日追踪明细-新_20190801.xlsx') # 方便操作直接excel载入数据集
                                # 加载mysql数据到内存
pd.set_option('display.max_columns', None) # 显示所有字段
df.head() #预览数据前五行

##  数据清洗

- 预览可知部分字段对当前分析无意义，需删除；

In [ ]:
df.shape # 查看原始行列数

In [ ]:
drop_columns = [ '渠道来源','是否申请验车','脱敏手机号','日利率','失效时间','是否按揭','订单状态','当前状态','邀约结果','派单时间','派单省份','派单城市','派单门店','预约到店时间','到店时间','提款成功时间','放款门店','当天通次', '当天接通通次', '当天有效通次', '当天通时', '近3日通次', '近3日接通通次', '近3日通时','近7日通次', '近7日接通通次', '近7日通时', '近15日通次', '近15日接通通次', '近15日通时', '本月累计通次', '本月累计接通通次', '本月累计通时']
df.drop(drop_columns,inplace=True,axis=1) # 删除字段；变量drop_columns为待删除字段的列表（list）
df.shape # 查看修改后行列数

- 预览可知数据集有缺失、空值，需处理；

In [ ]:
df.replace('',np.nan,inplace=True) # 将空格值替换成np.nan统一进行空值处理；
df.replace(r'\s+', np.nan, regex=True,inplace=True)# 将空格值替换成np.nan统一进行空值处理；参数regex=True启用正则；

注：基于数据集理解可把所有空值简单替换成数值0；具体数据集具体分析空值处理

In [ ]:
df = df.fillna(0) # 将所有空值替换为数值0；

In [ ]:
df.rename(columns={'数据累计通时  ': '数据累计通时'},inplace=True) # 修改字段名；字段名含有空格

- 预览可知数据集有异常值，需处理；

In [ ]:
df[((df['放款金额']<20000)&(df['放款金额']>0))|((df['数据累计通次']<0)|(df['数据累计接通通次']<0)|(df['数据累计接通通次']<0)|(df['数据累计有效通次']<0)|(df['数据累计通时']<0))]
# 检测异常值

In [ ]:
df.loc[df['流水号']=='201901010447544001A','数据累计通时'] = 45 # 将异常值赋值为正常值

- 防止数据集'流水号'字段有重复值，需处理；

In [ ]:
df.shape # 查看原始行列数

In [ ]:
df.drop_duplicates(subset='流水号') # 删除含有重复值的行，基于‘ID’字段；参数subset指定基于哪个字段删除重复行
df.shape # 查看修改后行列数

- 重设索引，索引为Pandas默认，需重设为‘流水号’字段为索引；注：索引唯一性，需先去重，空、缺失值处理；

In [ ]:
df.set_index('流水号',inplace=True) # 更改索引

- 防止str类型值中含有空格，需处理

In [ ]:
str_columns = ['所在省', '所在市', '所在地区', '省', '市', '姓名', '坐席名字','小组名称', '部门名称']
# 需要去除str类型值含有

for i in str_columns: # fro循环遍历去除str类型值中含有的空格
    df['%s'%(i)].str.strip()

- 查看字段的数据类型，合理更正。

In [ ]:
df.info() # 查看数据集基本信息

In [ ]:
date_colums = ['申请时间','分配时间','门店退回时间','门店提交初审时间','终审通过时间'] # 数据类型待转换为时间类型的字段

for i in date_colums: # for循环 批量修改
    df['%s'%(i)] = pd.to_datetime(df['%s'%(i)], format ='%Y-%m-%d') # 数据类型转换为时间类型
    
df['数据累计通时'] = df['数据累计通时'].astype('float64') # 数据类型需要转换为浮点数的字段

- 合并两个重复字段，以最后字段为基准；

In [ ]:
merge_index = list(df[df['省'] == 0]['省'].index) # 确定‘省’字段空缺值的索引，生成一个可迭代对象list

for i in merge_index: 
    df['省']['%s'%(i)] = df.loc['%s'%(i),'所在省'] # 遍历合并‘省’字段 和 ‘所在省’字段；以‘省’字段为基准
    
merge_index = list(df[df['市'] == 0]['市'].index) # 确定‘市’字段空缺值的索引，生成一个可迭代对象list

for i in merge_index: 
    df['市']['%s'%(i)] = df.loc['%s'%(i),'所在市'] # 遍历合并‘市’字段 和 ‘所在市’字段；以‘市’字段为基准

D:\WD53259\Install\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\WD53259\Install\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


注：此方法运行效率低下，寻求优化中；（pandas会报警告）

In [ ]:
df[(df['省'] == 0)&(df['所在省'] == 0)] #查看合并后数据集
#df[(df['省'] == 0)&(df['所在省'] != 0)] #验证合并后数据集

In [ ]:
df[(df['市'] == 0)&(df['所在市'] == 0)] #查看合并后数据集
#df[(df['市'] == 0)&(df['所在市'] != 0)] #验证合并后数据集

# 二、数据探索

In [48]:
df

,所在省,所在市,所在地区,省,市,姓名,借款人uid,初始额度,申请时间,分配时间,坐席名字,坐席uid,小组名称,部门名称,门店退回时间,门店提交初审时间,终审通过时间,终审额度,放款金额,数据累计通次,数据累计接通通次,数据累计有效通次,数据累计通时
流水号,,,,,,,,,,,,,,,,,,,,,,,
1190723101839560139,新疆,阿拉尔,0,新疆,0,王雅萍,17042563,76051.0,1970-01-01,1970-01-01,郑姝博,14775190,一组,贷款业务一部,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,12.0,6.0,6.0,407.0
201904160577116317A,浙江省,温州市,永嘉县,浙江省,温州市,曾玲,20198848,197000.0,1970-01-01,1970-01-01,吴国华,11720848,一组,贷款业务一部,1970-01-01,1970-01-01,1970-01-01,171000.0,171000.0,23.0,19.0,17.0,2287.0
201903050531599377A,浙江省,杭州市,余杭区,浙江省,温州市,秦邱涛,10711585,68000.0,1970-01-01,1970-01-01,朱容辉,13480573,五组,贷款业务三部,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,4.0,4.0,3.0,125.0
201810280374949836A,浙江省,温州市,鹿城区,浙江省,温州市,张卡迪,16469543,68000.0,1970-01-01,1970-01-01,王惟潞,14775574,一组,贷款业务一部,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,12.0,7.0,7.0,765.0
1190326101167045641,新疆,阿拉尔,0,新疆,0,龙仕武,17664903,39200.0,1970-01-01,1970-01-01,许克波,14889353,二组-借呗客服,客服,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,12.0,0.0,0.0,0.0
201810020338024949A,安徽省,蚌埠市,蚌山区,安徽省,蚌埠市,葛俊杰,16014449,66000.0,1970-01-01,1970-01-01,郑姝博,14775190,一组,贷款业务一部,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,10.0,6.0,5.0,395.0
11901019777830132,新疆,阿拉尔,0,新疆,0,陈怀波,15949628,44250.0,1970-01-01,1970-01-01,俞嘉杰,14889211,二组-借呗客服,客服,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,14.0,1.0,1.0,328.0
201904030576672840A,吉林省,长春市,朝阳区,吉林省,长春市,查洪亮,20877805,52000.0,1970-01-01,1970-01-01,马靖鸿,26003823,五组,贷款业务三部,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,2.0,1.0,1.0,372.0
201809280313490482A,河南省,郑州市,金水区,河南省,商丘市,张昌华,15930346,59000.0,1970-01-01,1970-01-01,谭田,3633350,五组,贷款业务三部,1970-01-01,1970-01-01,1970-01-01,0.0,0.0,2.0,2.0,2.0,99.0
